# HEP in the browser using JupyterLite and Emscripten-forge¶

- My name's Angus
- I am a Postdoc/Senior Research Assistant at Princeton (depending upon the contract!)
- Working on the Awkward Array project and wider ecosystem
- My interests are in "holistic software development" (a term I just made up) i.e. everything from CI/CD, to documentation.
- I've recently been interacting more with building applications for the web, and thinking about some of the challenges associated with HEP.

## Running Python in the Browser

Over the years, many projects have attempted to bring Python to the web-browser

- Skulpt
- PyPy.js
- Brython
- Transcrypt

These efforts fall into two categories:

|                          Transpiler                          	|                      Interpreter                      	|
|:------------------------------------------------------------:	|:-----------------------------------------------------:	|
| User program is _transpiled_ from Python to JS ahead of time 	| User program is _interpreted_ by a JS-based Python VM 	|

Brython is a Python transpiler. Consider the following test program:
:::{code} python
:name: brython-snippet
:caption: Simple Python program to transiple with Brython.

for i in range(10):
    print(i)
:::

For this simple snippet, Brython generates the following JavaScript:
:::{code} javascript
:caption: Generated JS for {numref}`brython-snippet`.

// Javascript code generated from ast
var $B = __BRYTHON__,
    _b_ = $B.builtins,
    locals___main__ = $B.imported["__main__"],
    locals = locals___main__,
    frame = ["__main__", locals, "__main__", locals]
frame.__file__ = '<string>'
locals.__name__ = '__main__'
locals.__doc__ = _b_.None
locals.__annotations__ = locals.__annotations__ || $B.empty_dict()
frame.$f_trace = $B.enter_frame(frame)
$B.set_lineno(frame, 1)

var _frames = $B.frames_stack.slice()
var stack_length = $B.frames_stack.length
try{
  frame.$lineno = 1
  var no_break_1389 = true,
      iterator_1389 = $B.$call(_b_.range, [9, 9, 18])(10)
  for(var next_1389 of $B.make_js_iterator(iterator_1389, frame, 1)){
    var v1390 = next_1389
    locals___main__.i = v1390
    $B.set_lineno(frame, 2);
    $B.$call(_b_.print, [1, 1, 9])(locals___main__.i)
  }
  $B.leave_frame({locals, value: _b_.None})
}catch(err){
  $B.set_exc(err, frame)
  if((! err.$in_trace_func) && frame.$f_trace !== _b_.None){
    frame.$f_trace = $B.trace_exception()
  }
  $B.leave_frame({locals, value: _b_.None})
  throw err
}
:::

PyPy.js is a version of the PyPy interpreter compiled for the web using [ASM.js](https://en.wikipedia.org/wiki/Asm.js), a subset of JavaScript that cooperating interpreters can optimise:

:::{code} javascript
:caption: An example of an ASM.js routine.
:name: asm-js-sample

function DiagModule(stdlib, foreign, heap) {
    "use asm";

    // Variable Declarations
    var sqrt = stdlib.Math.sqrt;

    // Function Declarations
    function square(x) {
        x = +x;
        return +(x*x);
    }

    function diag(x, y) {
        x = +x;
        y = +y;
        return +sqrt(square(x) + square(y));
    }

    return { diag: diag };
}

:::

These efforts solved the problem of running "arbitrary" Python code in the browser.

But, most of the HEP ecosystem uses Python extension modules, e.g. NumPy, SciPy, Awkward Array, Boost.Histogram, etc. How can we run those on the web?

:::{image} img/one-ring-meme-small.png
:align: center
:width: 100%

:::

Emscripten is
> [...] a complete compiler toolchain to WebAssembly, using LLVM [...]

It can e.g. compile a trivial hello world in C ({numref}`hello-world`)
:::{code} c
:caption: A simple C++ hello-world program.
:name: hello-world

#include <stdio.h>

int main() {
  printf("hello, world!\n");
  return 0;
}
:::

to a a web-friendly JS module, using a simple CLI ({numref}`emcc-cli`)
:::{code} bash
:caption: Building an Emscripten-powered REPL with `emcc`.
:name: emcc-cli

# Compile
emcc -c test.c -o test.o
# Link
emcc test.o -o test.html
:::

## Building an Ecosystem

Just being able to run arbitrary code on the web is not enough.

Modern software is composable, collaborative, and complex!

We need to be able to build a software ecosystem.

:::{figure} img/pyodide-logo-readme.png
:align: left

Pyodide logo.
:::

It facilitates interfacing between JS and Python using FFI
:::{code} javascript
:caption: Listening for events from Python.

from pyodide.ffi import create_proxy
from js import document

# Create callback
def my_callback():
    print("hi")
proxy = create_proxy(my_callback)

# Add event listener!
document.body.addEventListener("click", proxy)
:::

Installing packages using `micropip`

:::{code} python
:caption: Using `micropip` in Pyodide.

import micropip
await micropip.install("matplotlib")

import matplotlib
:::

including pure-Python wheels from PyPI, *and* over 200 other compiled packages!

If you've heard of `<pyscript>`, it's Pyodide that powers it!

Adding a package to Pyodide involves creating a recipe in their repository (see {numref}`pyodide-recipe`):
:::{code} yaml
:caption: `awkward-cpp` Pyodide build recipe.
:name: pyodide-recipe

package:
  name: awkward-cpp
  version: 22
  top-level:
    - awkward_cpp

source:
  url: https://files.pythonhosted.org/packages/8b/f8/97efcc2d52e2f9967f9beed86ed6736fc6eed91eb2d974a1bdc8c438146d/awkward-cpp-22.tar.gz
  sha256: 21679636fb21cfe3715f88a32326a579199384db2da4a62995c310502d7fe85f

build:
  script: |
    export CMAKE_ARGS="${CMAKE_ARGS} -DEMSCRIPTEN=1"
  exports: whole_archive

requirements:
  run:
    - numpy

test:
  imports:
    - awkward_cpp

about:
  home: https://pypi.org/project/awkward-cpp/
  summary: CPU kernels and compiled extensions for Awkward Array
  license: BSD-3-Clause
:::

:::{figure} img/pyodide-logo-readme-sad.png
:align: left

An unhappy Pyodide logo.
:::

Pyodide is not perfect!

One can't pre-install wheels without rebuilding entire Pyodide distribution

Packages are built as a _distribution_, meaning no choices in version

It uses custom (one-off) tooling to build releases

:::{figure} img/winnie-pooh-meme.png
:align: left

A technical representation of Pyodide vs emscripten-forge.
:::

Emscripten-forge is a _community-maintained_ distribution, which crowdsources maintainership through centralised channels.

It leverages the existing tooling around conda/mamba

:::{code} yaml
:caption: The Conda environment for this talk's demo.
:name: environment-demo

name: xeus-python-kernel
channels:
  - https://repo.mamba.pm/emscripten-forge
  - https://repo.mamba.pm/conda-forge
dependencies:
  - awkward >=2.4.0
  - hist-base
  - vector
  - numpy
  - matplotlib-base
  - mplhep
  - uproot-base >=5.0.12
:::

It supports pinning package versions (good for reproducibility)

Packages can be pre-installed, and their contents pruned!

## After ASM.js

Software engineers decided that things were too simple with ASM.js

They came up with their own stack-based virtual machine to execute a new bytecode format called [Web Assembly](https://en.wikipedia.org/wiki/WebAssembly)

Emscripten can compile existing C++ programs and generate web assembly

:::{image} img/mind-blown.gif
:::

## Running _Analyses_ in the Browser

Running Python code in a web-page is cool! But, most of us don't write applications.

Scientists 👩‍🔬 use Jupyter (Lab), though many still use Jupyter Notebook (see {numref}`jupyter-notebook`).

:::{figure} img/jupyterpreview.png
:align: left
:name: jupyter-notebook

Jupyter Notebook User Interface.
:::

:::{note} 
This _talk_ is written in JupyterLab! <button data-commandlinker-command="deck:stop">Click here to see</button>
:::

:::{image} img/jupyter-lab-dark.svg
:::

JupyterLab is built in TypeScript (JavaScript)

It is composed from the ground up of from _plugins_ that provide core functionality

It is able to communicate with remote servers that execute user code

Running JupyterLab requires a web-server for file IO and executing kernels

:::{image} img/my-binder.png
:::

<https://mybinder.org> provides a service to build and serve Jupyter sessions to anonymous users

Supports live demonstrations, but also underpins reproducibility efforts — [![badge](https://mybinder.org/badge_logo.svg)](mybinder.org)

Limited resources struggles to meet significant demand

:::{image} img/my-binder-blog.png
:width: 100%
:::

:::{image} img/jupyterlite.webp
:align: center
:width: 250px
:::

_JupyterLite_ is an entirely in-browser version of JupyterLab. 

It reuses most of JupyterLab's extensions, with some additional plugins for kernels etc.

Like JupyterLab, it supports RTC, File Browsers, and much more!

The `jupyterlite-pyodide-kernel` implements a Python kernel using Pyodide.

It provides `piplite`, to allow pre-registration of custom wheels.

:::{code} ipython
:caption: Example piplite command

%piplite install awkward
:::

:::{image} img/xeus-python.svg

:::

In practice, the `xeus-python` kernel tends to be faster than `ipykernel`

`xeus-python` has near feature parity with `ipykernel`

`jupyterlite/xeus-python-kernel` supports bundling packages from an `environment.yml` (see {numref}`environment-demo`)

:::{card} Go to Demo
:link: demo.ipynb

A demo of the [PyHEP 2022 Talk by Jim Pivarski](https://indico.cern.ch/event/1150631/contributions/5000596/).
:::